<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

## Reddit API:
1. Go to https://www.reddit.com/prefs/apps
2. Click "create application"
3. Name: My Sentiment Example
4. Choose script
5. Leave *description* and *about url* blank
6. Add http://www.example.com/unused/redirect/uri to *redirect uri*

In [17]:
%pip install praw


[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Imports

In [18]:

import praw
import os
import openai
from dotenv import load_dotenv
load_dotenv()

True

In [19]:
reddit = praw.Reddit(client_id=os.environ.get('client_id'), # the App ID retreived from the reddit application
                     client_secret=os.environ.get('client_secret'), # the secret retreived from the reddit application
                     user_agent="sentiment analysis test (by u/YourUserName)"  # add your reddit name
                    )

We can now already use our reddit instance to obtain public information from reddit!

In [20]:
for submission in reddit.subreddit("test").hot(limit=10):
    print(submission.title)


test
testy mcface
Test post wait can’t I put words in ?
Link Submission
Link Submission
Link Submission
My Link Submission
My Link Submission
My Link Submission
My Link Submission


Of course we want to access hot subreddits like r/wallstreetbets or r/stocks

In [21]:
subreddit_stocks = reddit.subreddit("stocks")

In [22]:
print(subreddit_stocks.display_name)
print(subreddit_stocks.title)
print(subreddit_stocks.accounts_active)

stocks
Stocks - Investing and trading for all
4205


Let's say we want to access the first 5 hot posts and three most upvoted comments:

In [23]:
for post in subreddit_stocks.hot(limit=5):
    print(post.title)
    submission = reddit.submission(post.id)
    c = 0
    for comment in submission.comments:
        comment = comment.body
        if not comment == "[deleted]":
            print(comment)
            c+=1
        if c == 2:
            break

Rate My Portfolio - r/Stocks Quarterly Thread March 2023
I am 26, so I have several decades until retirement. My goal is to hold the stocks in my portfolio perpetually, though I'm not reserved to make adjustments and sell when a) I see an opportunity to take profits b) my thesis changes. My long term goal is passive income generation, though I don't chase yields. I don't pay much attention to valuation or macro, as I think that since I plan on holding these stocks for decades while buying every month, those don't matter to me much. I invest regularly every month, so I'm okay with buying at a higher valuation at times and lower valuation at other times. In my opinion, buying at the best possible time is more difficult than buying a good company. That said, if I see an undervalued stock I like, I don't hesitate to buy more and when I make my regular contributions, I try weigh towards stocks that might have been beaten down or seem undervalued. My last review in one of these was [Dec 2021

We can now use GPT to extract stock tickers from all of these posts and tell us whether the sentiment for the mentioned assets is positive, neutral or negative:

## OpenAI API

Remember to use the notebook as shown, you must set your OpenAI API Key as an environment variable. Obviously, there are many ways you could provide your API Key to the Python code, input() or even hard-coded, but those are typically not recommended for safety reasons. Having it as an environment variable let's the key live on the computer, but not actually be present in the code.

### Set-up Open AI API Key

We'll only need to do this once per computer

In [24]:
# Uncomment below and swap in your key to place your environment key using Python
# Then you can delete the key string and the code cell below will still work!
# os.environ["OPENAI_API_KEY"] = "Your key goes here!

In [25]:
openai.api_key = os.getenv("OPENAI_API_KEY")

## Prompt Generation
First, we need to create a dictionary containing the title and top three comments

In [26]:
getattr(subreddit_stocks, "hot")

<bound method BaseListingMixin.hot of Subreddit(display_name='stocks')>

In [27]:
def get_titles_and_comments(subreddit="stocks", sub_instance="hot", limit=10, num_comments=2, skip_first=2):
    subreddit = reddit.subreddit(subreddit)
    titles_and_comments = {}
    for c, post in enumerate(getattr(subreddit, sub_instance)(limit=limit)):
        
        if c < skip_first:
            continue
        
        c+=(1-skip_first)
        
        titles_and_comments[c] = ""

        submission = reddit.submission(post.id)
        title = post.title
        
        titles_and_comments[c] += "Title: " + title  + "\n\n"
        titles_and_comments[c] += "Comments: \n\n"
        
        comment_counter = 0
        for comment in submission.comments:
            comment = comment.body
            if not comment == "[deleted]":
                titles_and_comments[c] += comment + "\n"
                comment_counter+=1
            if comment_counter == num_comments:
                break

    return titles_and_comments

In [28]:
titles_and_comments = get_titles_and_comments(subreddit="stocks", limit=12)

In [29]:
print(titles_and_comments[2])

Title: If you have $50k to invest into VOO how would you do so?

Comments: 

Welcome to r/stocks!

For stock recommendations please see our portfolio sticky, sort by hot, it's the first sticky, or see [past portfolio stickies here.](https://www.reddit.com/r/stocks/search?q=author%3Aautomoderator+title%3A%22Rate+My+Portfolio%22&restrict_sr=on&sort=new&t=all)

For beginner advice, brokerage info, book recommendations, even advanced topics and more, please read our [Wiki here.](https://www.reddit.com/r/stocks/wiki/index)

If you're wondering **why a stock moved** a certain way, check out [Finviz](https://finviz.com/quote.ashx?t=spy) which aggregates the most news for almost every stock, but also see [Reuters](https://www.reuters.com/), and even [Yahoo Finance](https://finance.yahoo.com/).

Also include *some* [due diligence](https://www.investopedia.com/terms/d/duediligence.asp) to this post or it may be removed if it's low effort.

*I am a bot, and this action was performed automatically

In [30]:
def create_prompt(title_and_comments):

    task = "Return the stock ticker or company in the following heading and comments and classify the sentiment. If no ticker or company is mentioned write 'No company mentioned':\n\n"
    return task + title_and_comments


In [31]:
prompt = create_prompt(titles_and_comments[1])

In [32]:
print(prompt)

Return the stock ticker or company in the following heading and comments and classify the sentiment. If no ticker or company is mentioned write 'No company mentioned':

Title: EU approves Microsoft’s $69 billion acquisition of Activision Blizzard, clearing major hurdle

Comments: 

Microsoft Excel Loot Crates confirmed
It's always a good time to buy more MSFT!



In [33]:
for key, title_and_comments in titles_and_comments.items():
    
    prompt = create_prompt(title_and_comments)
    
    response = openai.Completion.create(engine="text-davinci-003",
                                                prompt=prompt,
                                                max_tokens=256,
                                                temperature=0,
                                                top_p=1.0,
                                                frequency_penalty=0.0,
                                                presence_penalty=0.0)
    print(title_and_comments)
    print("Sentiment: " + response["choices"][0]["text"])
    print("-"*30)

Title: EU approves Microsoft’s $69 billion acquisition of Activision Blizzard, clearing major hurdle

Comments: 

Microsoft Excel Loot Crates confirmed
It's always a good time to buy more MSFT!

Sentiment: 
Company: Microsoft (MSFT)
Sentiment: Positive
------------------------------
Title: If you have $50k to invest into VOO how would you do so?

Comments: 

Welcome to r/stocks!

For stock recommendations please see our portfolio sticky, sort by hot, it's the first sticky, or see [past portfolio stickies here.](https://www.reddit.com/r/stocks/search?q=author%3Aautomoderator+title%3A%22Rate+My+Portfolio%22&restrict_sr=on&sort=new&t=all)

For beginner advice, brokerage info, book recommendations, even advanced topics and more, please read our [Wiki here.](https://www.reddit.com/r/stocks/wiki/index)

If you're wondering **why a stock moved** a certain way, check out [Finviz](https://finviz.com/quote.ashx?t=spy) which aggregates the most news for almost every stock, but also see [Reuters](